### import bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup

### Wysłanie żądania dostępu do strony internetowej z opiniami o produkcie

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews"
response = requests.get(url)
response.status_code


### Jeżeli operacja zakończy się powodzeniem, wyodrębnienie z kodu strony opinii o produkcie 

In [ ]:
if response.status_code == 200:
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review:not(.user-post--highlight)")

### Dla każdej opinii wyoądrebnienie z kodu html poszczególnych składowych i przypisanie ich do elementów złożonej struktury danych

In [ ]:
for opinion in opinions:
    try:
        single_opinion = {
            "author": opinion.select_one("span.user-post__author-name").text.strip(),
            "recommendation": opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
            "stars": opinion.select_one("span.user-post__score-count").text.strip(),
            "content": opinion.select_one("div.user-post__text").text.strip(),
            "prons": [p.text.strip() for p in opinion.select_one("div.review-feature__item--positive")],
            "cons": [c.text.strip() for c in opinion.select_one("div.review-feature__item--negative")],
            "usefull": opinion.select_one("button.vote-yes > span").text.strip(),
            "useless": opinion.select_one("button.vote-no > span").text.strip(),
            "post_date": opinion.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip(),
            "purchase_date": opinion.select_one("span.user-post__published > time:nth-child(2)")['datetime'].strip(),
    }
        print(single_opinion)
    except (AttributeError, TypeError):
        pass
    